In [1]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print(train_data.shape)
print(test_data.shape)

57026/57026 [==============================] - 0s 1us/step
(404, 13)
(102, 13)


In [3]:
#data normalization

mean = train_data.mean(axis=0)
train_data -= mean

std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

In [4]:
from keras import models
from keras import layers
# function to build models easy modularity purposes
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [5]:
#K-fold validation

import numpy as np
k=4
num_val_samples = len(train_data) // k   #// is the  division operator, if total 404 then here 404/4 is 101.
num_epochs = 100
all_scores = []


for i in range(k):
    print('Processing Fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate([train_data[:i * num_val_samples],
                                        train_data[(i + 1) * num_val_samples:]],
                                        axis=0)
    
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],
                                           train_targets[(i + 1) * num_val_samples:]],
                                           axis=0)
    
    model = build_model()   # call function
    
    # using a batch size of 1 (i.e., stochastic gradient descent). 
    #The verbose=0 argument is used to suppress the output of training progress during each epoch.
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

Processing Fold # 0
Processing Fold # 1
Processing Fold # 2
Processing Fold # 3


In [6]:
print(all_scores)
print(np.mean(all_scores))

[2.148374557495117, 2.7199628353118896, 2.604037046432495, 2.541865348815918]
2.503559947013855


In [7]:
num_epochs = 300
all_mae_histories = []


for i in range(k):
    print('Processing Fold #', i)
    
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate([train_data[:i * num_val_samples],
                                        train_data[(i + 1) * num_val_samples:]],
                                        axis=0)

    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],
                                           train_targets[(i + 1) * num_val_samples:]],
                                           axis=0)
    
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                      validation_data=(val_data, val_targets),
                      epochs=num_epochs, batch_size=1, verbose=0)
    
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

Processing Fold # 0
Processing Fold # 1
Processing Fold # 2
Processing Fold # 3


In [8]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
print(test_mae_score)

4/4 [==============================] - 0s 5ms/step - loss: 18.3182 - mae: 2.7423
2.7422776222229004
